# Sumarização com similaridade do cosseno

## Preparação do texto de exemplo

In [1]:
# importação das bibliotecas utilizadas
import re # expressão regular
import nltk # linguagem natural
import string # texto
import numpy as np # matemática
import networkx as nx # gráfos
from nltk.cluster.util import cosine_distance # parte do cosseno da nltk

In [2]:
# Baixando os módulos do nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Pegando as stopwords em português
stopwords = nltk.corpus.stopwords.words('portuguese')
print(stopwords)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [5]:
# Função de processamento do texto, tiram pontuação, letras maiúsculas, números ...
def preprocessamento(texto):
  texto_formatado = texto.lower()
  tokens = []
  for token in nltk.word_tokenize(texto_formatado):
    tokens.append(token)

  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

  return texto_formatado

In [7]:
# Texto de exemplo aplicando a formatação adequada
texto_original = """A inteligência artificial é a inteligência similar à humana máquinas.
                    Definem como o estudo de agente artificial com inteligência.
                    Ciência e engenharia de produzir máquinas com inteligência.
                    Resolver problemas e possuir inteligência.
                    Relacionada ao comportamento inteligente.
                    Construção de máquinas para raciocinar.
                    Aprender com os erros e acertos.
                    Inteligência artificial é raciocinar nas situações do cotidiano."""
texto_original = re.sub(r'\s+', ' ', texto_original)
texto_original

'A inteligência artificial é a inteligência similar à humana máquinas. Definem como o estudo de agente artificial com inteligência. Ciência e engenharia de produzir máquinas com inteligência. Resolver problemas e possuir inteligência. Relacionada ao comportamento inteligente. Construção de máquinas para raciocinar. Aprender com os erros e acertos. Inteligência artificial é raciocinar nas situações do cotidiano.'

## Função para calcular a similaridade entre sentenças

- Link: https://en.wikipedia.org/wiki/Cosine_similarity
- Cálculos passo a passo: https://janav.wordpress.com/2013/10/27/tf-idf-and-cosine-similarity/

In [8]:
# Pegando as sentenças originais e as formatadas, como já visto
sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto_original)]
sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]

In [9]:
sentencas_originais

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [10]:
sentencas_formatadas

['inteligência artificial inteligência similar humana máquinas',
 'definem estudo agente artificial inteligência',
 'ciência engenharia produzir máquinas inteligência',
 'resolver problemas possuir inteligência',
 'relacionada comportamento inteligente',
 'construção máquinas raciocinar',
 'aprender erros acertos',
 'inteligência artificial raciocinar situações cotidiano']

In [11]:
# Função para calcular uma nota de quanto duas sentenças se parecem entre si
def calcula_similaridade_sentencas(sentenca1, sentenca2):
  # Tokeniza as falavras de cada sentença
  palavras1 = [palavra for palavra in nltk.word_tokenize(sentenca1)]
  palavras2 = [palavra for palavra in nltk.word_tokenize(sentenca2)]

  # Junta as palavras únicas das duas sentenças
  todas_palavras = list(set(palavras1 + palavras2))

  # Iniciando o vetor de cada sentença no tamanho de todas as palavras com zero
  vetor1 = [0] * len(todas_palavras)
  vetor2 = [0] * len(todas_palavras)

  # Adiciona 1 no vetor de palavra que pertence ao conjunto de cada sentença
  for palavra in palavras1:
    vetor1[todas_palavras.index(palavra)] += 1
  for palavra in palavras2:
    vetor2[todas_palavras.index(palavra)] += 1

  # Retorna calculando o cosseno entre os valores nos vetores
  return 1 - cosine_distance(vetor1, vetor2)

In [21]:
calcula_similaridade_sentencas(sentencas_formatadas[0], sentencas_formatadas[0])

0.9999999999999998

## Função para gerar a matriz de similaridade

In [40]:
# Função para calular a similaridade de todas com todas as sentenças
def calcula_matriz_similaridade(sentencas):
  matriz_similaridade = np.zeros((len(sentencas), len(sentencas)))

  # Faz para cada sentença o cálculo de similarizade, SIM repete-se os valores dos triagulares superior e inferior, mas não na diagonal principal
  for i in range(len(sentencas)):
    for j in range(len(sentencas)):
      if j == i:
        continue
      matriz_similaridade[i][j] = calcula_similaridade_sentencas(sentencas[i], sentencas[j])

  return matriz_similaridade

In [41]:
calcula_matriz_similaridade(sentencas_formatadas)

array([[0.        , 0.47434165, 0.47434165, 0.35355339, 0.        ,
        0.20412415, 0.        , 0.47434165],
       [0.47434165, 0.        , 0.2       , 0.2236068 , 0.        ,
        0.        , 0.        , 0.4       ],
       [0.47434165, 0.2       , 0.        , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.2       ],
       [0.35355339, 0.2236068 , 0.2236068 , 0.        , 0.        ,
        0.        , 0.        , 0.2236068 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.20412415, 0.        , 0.25819889, 0.        , 0.        ,
        0.        , 0.        , 0.25819889],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.47434165, 0.4       , 0.2       , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.        ]])

## Função para sumarizar

- Algoritmo Page Rank: https://www.youtube.com/watch?v=YfDNI1jp5sM e https://www.youtube.com/watch?v=YplmCue8XJU

In [50]:
# Função de sumarização do texto, já visto em outras vezes, mas agora como o cálculo do cosseno usando a matriz se similaridade
def sumarizar(texto, quantidade_sentencas):
  # Separa o texto em sentenças e formata cada uma delas para depois criar a matriz de similaridade
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]
  matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)

  # Criando do grafo a partir da matriz de similaridade
  grafo_similaridade = nx.from_numpy_array(matriz_similaridade)

  # Usa o algoritmo de PageRank para calcular as notas
  notas = nx.pagerank(grafo_similaridade)

  # Ordena reversamente as notas obtidas, do maior para o menor
  notas_ordenadas = sorted(((notas[i], nota) for i, nota in enumerate(sentencas_originais)), reverse=True)

  # Faz a seleção das melhores sentenças
  melhores_sentencas = []
  for i in range(quantidade_sentencas):
    melhores_sentencas.append(notas_ordenadas[i][1])

  return sentencas_originais, melhores_sentencas, notas_ordenadas

In [45]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(texto_original, 3)

In [46]:
sentencas_originais

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [47]:
melhores_sentencas

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.',
 'Ciência e engenharia de produzir máquinas com inteligência.']

In [48]:
notas_sentencas

[(0.22820924040178003,
  'A inteligência artificial é a inteligência similar à humana máquinas.'),
 (0.1839190802151221,
  'Inteligência artificial é raciocinar nas situações do cotidiano.'),
 (0.1633191450783496,
  'Ciência e engenharia de produzir máquinas com inteligência.'),
 (0.1543717033327776,
  'Definem como o estudo de agente artificial com inteligência.'),
 (0.12639273963873288, 'Resolver problemas e possuir inteligência.'),
 (0.09616903563964783, 'Construção de máquinas para raciocinar.'),
 (0.023809527846794958, 'Relacionada ao comportamento inteligente.'),
 (0.023809527846794958, 'Aprender com os erros e acertos.')]

## Visualização do resumo

In [57]:
from IPython.core.display import HTML

# Função para formatação e vizualização do texto destacando as melhores sentenças
def visualiza_resumo(titulo, lista_sentencas, melhores_sentencas):
  texto = ''
  display(HTML(f'<h1>Resumo do texto - {titulo}</h1>'))

  for sentenca in lista_sentencas:
    if sentenca in melhores_sentencas:
      texto += f"<mark>{sentenca}</mark>"
    else:
      texto += sentenca

  display(HTML(f""" {texto} """))

In [58]:
visualiza_resumo('Teste', sentencas_originais, melhores_sentencas)

## Extração de texto da internet

In [59]:
!pip install goose3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.7 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=ddc740179a44b42471440e0492ff3ab47dc702456f764e73fb74d8b0537c91ca
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [60]:
# Importando e criando o objeto para extrair textos da web
from goose3 import Goose
g = Goose()
url = 'https://iaexpert.academy/2020/11/09/ia-preve-resultado-das-eleicoes-americanas/'
artigo = g.extract(url)

In [68]:
artigo.cleaned_text

'Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton. Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos. Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados. Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.\n\nO modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais. O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito. No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.\n\nO Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano passado. Desta vez, o mode

In [63]:
# Aplica a sumarização nesse texto
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(artigo.cleaned_text, 5)

In [64]:
sentencas_originais

['Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton.',
 'Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos.',
 'Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados.',
 'Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.',
 'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.',
 'O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito.',
 'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.',
 'O Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano pas

In [65]:
melhores_sentencas

['Desta vez, o modelo está treinando com cerca de 5 vezes mais dados que nas eleições americanas anteriores.',
 'Quando seu modelo foi usado para predizer os resultados da eleição corrente usando dados brutos, Joe Biden apareceu como vencedor com larga vantagem.',
 'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.',
 'Segundo o Dr. Makse, integrar estas duas variáveis em seu modelo é a parte mais importante do trabalho.',
 'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.']

In [66]:
notas_sentencas

[(0.10322707053208108,
  'Desta vez, o modelo está treinando com cerca de 5 vezes mais dados que nas eleições americanas anteriores.'),
 (0.08467351062155114,
  'Quando seu modelo foi usado para predizer os resultados da eleição corrente usando dados brutos, Joe Biden apareceu como vencedor com larga vantagem.'),
 (0.07824901512415162,
  'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.'),
 (0.07627442688432466,
  'Segundo o Dr. Makse, integrar estas duas variáveis em seu modelo é a parte mais importante do trabalho.'),
 (0.07333552035097592,
  'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.'),
 (0.07240389589770964,
  'Parece que, desta vez, os algoritmos estão de fato contribuindo para que as predições sejam mais precisas.'),
 (0.06666298135659274,
  'O Dr. Makse disse que seu trabalho 

In [69]:
visualiza_resumo(artigo.title, sentencas_originais, melhores_sentencas)

## Solução para o exercício - lematização

In [73]:
# Importa a biblioteca para trabalhar com vizualização e formatação de texto
import spacy

In [74]:
!python -m spacy download pt_core_news_sm

2023-12-27 00:00:45.300114: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-27 00:00:45.300201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-27 00:00:45.302254: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-27 00:00:47.599623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 62.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [82]:
# Baixa os módulos em português
pln = spacy.load("pt_core_news_sm")
pln

In [76]:
# Função já implementada anteriormente de lematização, agora usando a biblioteca spacy
def preprocessamento_lematizacao(texto):
  texto = texto.lower()
  texto = re.sub(r" +", ' ', texto)

  documento = pln(texto)
  tokens = []
  for token in documento:
    tokens.append(token.lemma_)

  tokens = [palavra for palavra in tokens if palavra not in stopwords and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

  return texto_formatado

In [85]:
# Função de lematização uzando os cálculos de cosseno, só que com o spacy
def sumarizar_lematizacao(texto, quantidade_sentencas):
  sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
  sentencas_formatadas = [preprocessamento_lematizacao(sentenca_original) for sentenca_original in sentencas_originais]
  matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)
  grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
  notas = nx.pagerank(grafo_similaridade)
  notas_ordenadas = sorted(((notas[i], nota) for i, nota in enumerate(sentencas_originais)), reverse=True)
  melhores_sentencas = []
  for i in range(quantidade_sentencas):
    melhores_sentencas.append(notas_ordenadas[i][1])

  return sentencas_originais, melhores_sentencas, notas_ordenadas

In [79]:
artigo.cleaned_text

'Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton. Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos. Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados. Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.\n\nO modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais. O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito. No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.\n\nO Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano passado. Desta vez, o mode

In [80]:
# Aplicando a sumarização original com o algoritmo anterior
sentencas_originais, melhores_sentencas, _ = sumarizar(artigo.cleaned_text, 5)
visualiza_resumo(artigo.title, sentencas_originais, melhores_sentencas)

In [84]:
# Aplicando a sumarização original com o algoritmo novo com cosseno
sentencas_originais, melhores_sentencas, _ = sumarizar_lematizacao(artigo.cleaned_text, 5)
visualiza_resumo(artigo.title, sentencas_originais, melhores_sentencas)